# 🔄 Augmentation 테스트 노트북

다양한 Augmentation을 시각적으로 테스트하는 노트북입니다.

## 사용 방법
1. 이 파일을 복사하여 `test_aug_본인이름.ipynb`로 저장
2. 다양한 augmentation 조합을 테스트


In [ ]:
import os
import sys
sys.path.append('..')

import numpy as np
import cv2
import matplotlib.pyplot as plt
import albumentations as A
from glob import glob


In [ ]:
# 샘플 이미지 로드
IMAGE_ROOT = "../../Segmentation/train/DCM"  # 실제 경로로 수정

image_files = glob(os.path.join(IMAGE_ROOT, "**/*.png"), recursive=True)
if len(image_files) > 0:
    sample_path = image_files[0]
    image = cv2.imread(sample_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    print(f"Loaded image shape: {image.shape}")


In [ ]:
# 테스트할 Augmentation 정의
transforms_dict = {
    "Original": None,
    "Resize": A.Resize(512, 512),
    "HorizontalFlip": A.HorizontalFlip(p=1.0),
    "Rotate15": A.Rotate(limit=15, p=1.0),
    "BrightnessContrast": A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=1.0),
    "ElasticTransform": A.ElasticTransform(p=1.0),
    "GridDistortion": A.GridDistortion(p=1.0),
    "GaussNoise": A.GaussNoise(var_limit=(10.0, 50.0), p=1.0),
}

# Augmentation 결과 시각화
fig, axes = plt.subplots(2, 4, figsize=(20, 10))
axes = axes.flatten()

for idx, (name, transform) in enumerate(transforms_dict.items()):
    if transform is None:
        aug_image = cv2.resize(image, (512, 512))
    else:
        resized = cv2.resize(image, (512, 512))
        aug = A.Compose([transform])
        aug_image = aug(image=resized)["image"]
    
    axes[idx].imshow(aug_image)
    axes[idx].set_title(name, fontsize=12)
    axes[idx].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
# 학습에 사용할 Augmentation 조합 테스트
train_transform = A.Compose([
    A.Resize(512, 512),
    A.Rotate(limit=15, p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.ElasticTransform(p=0.3),
])

# 같은 이미지에 여러번 적용해서 결과 확인
fig, axes = plt.subplots(2, 4, figsize=(20, 10))
axes = axes.flatten()

for idx in range(8):
    if idx == 0:
        aug_image = cv2.resize(image, (512, 512))
        title = "Original"
    else:
        aug_image = train_transform(image=image)["image"]
        title = f"Aug #{idx}"
    
    axes[idx].imshow(aug_image)
    axes[idx].set_title(title)
    axes[idx].axis('off')

plt.suptitle("Training Augmentation Results", fontsize=14)
plt.tight_layout()
plt.show()
